# Modelo exato de linhas de distribuição

In [1]:
import numpy as np
%precision 4

u'%.4f'

In [2]:
from numeric.util import P as p
from numeric.util import R as r

Uma carga equilibrada com potência de 6.000 kVA, tensão nominal de 12,47kV e fator de potência 0,9 indutivo está conectada no nó m de uma linha de distribuição trifásica de 10.000 pés (3,048 km) conforme mostrado na figura, abaixo:

<img src="Figuras/fig_032.png" width="600"/>

A configuração da linha está mostrada na figura abaixo:

<img src="Figuras/fig_021.png" width="400"/>

Os condutores utilizados são:

336,400 26/7 ACSR (com ampacidade de 530A) nas fases e 4/0 6/1 ACSR no neutro (com ampacidade de 340A).

Determine as matrizes **a**, **b**, **c**, **d**.

Usando as matrizes determine as tesões no nó fonte *n* e as correntes que fluem na linha.

## Definição dos parâmetros de entrada

Das questões anteriores, podemos obter a matriz de impedância de fase:

In [3]:
zabc = np.array([[ 0.4576+1.078j ,  0.1559+0.5017j,  0.1535+0.3849j],
                 [ 0.1559+0.5017j,  0.4666+1.0482j,  0.1580+0.4236j],
                 [ 0.1535+0.3849j,  0.1580+0.4236j,  0.4615+1.0651j]])
print zabc

[[ 0.4576+1.078j   0.1559+0.5017j  0.1535+0.3849j]
 [ 0.1559+0.5017j  0.4666+1.0482j  0.1580+0.4236j]
 [ 0.1535+0.3849j  0.1580+0.4236j  0.4615+1.0651j]]


E a matriz de admitância em derivação:

In [4]:
yabc = np.array([[ 0.+5.6711j, -0.-1.8362j, -0.-0.7033j],
              [-0.-1.8362j,  0.+5.9773j, -0.-1.1689j],
              [-0.-0.7033j, -0.-1.1689j,  0.+5.391j ]])
print yabc

[[ 0.+5.6711j -0.-1.8362j -0.-0.7033j]
 [-0.-1.8362j  0.+5.9773j -0.-1.1689j]
 [-0.-0.7033j -0.-1.1689j  0.+5.391j ]]


Multiplicando pelo comprimento da linha é possível obtermos a matriz de impedância e admitância total:

In [5]:
Zabc = 10e3 / 5280.0 * zabc
Zabc

array([[ 0.8667+2.0417j,  0.2953+0.9502j,  0.2907+0.729j ],
       [ 0.2953+0.9502j,  0.8837+1.9852j,  0.2992+0.8023j],
       [ 0.2907+0.729j ,  0.2992+0.8023j,  0.8741+2.0172j]])

In [6]:
Yabc = 10e3 / 5280.0 * yabc
Yabc

array([[ 0.+10.7407j,  0. -3.4777j,  0. -1.332j ],
       [ 0. -3.4777j,  0.+11.3206j,  0. -2.2138j],
       [ 0. -1.332j ,  0. -2.2138j,  0.+10.2102j]])

É possível observar que os elementos da matriz Yabc são bem pequenos pois essa matriz é dada em $\mu S$.

## Cálculo das matrizes generalizadas

Calculando agora as matrizes generalizadas:

\begin{eqnarray}
        \mathbf{a} & = & \mathbf{U} + \frac{1}{2} \cdot \mathbf{Z_{abc}} \cdot \mathbf{Y_{abc}} \\
        \mathbf{b} & = & \mathbf{Z_{abc}} \\
        \mathbf{c} & = & \frac{1}{2} \cdot \mathbf{Y_{abc}} \left[ \mathbf{U} + \mathbf{a} \right] \\
        \mathbf{d} & = & \mathbf{U} + \frac{1}{2} \cdot \mathbf{Y_{abc}} \cdot \mathbf{b}
\end{eqnarray}

In [7]:
U = np.identity(3)
U

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [8]:
a = U + 0.5 * np.dot(Zabc, Yabc*1e-6)
a.round(2)

array([[ 1.+0.j, -0.-0.j, -0.+0.j],
       [-0.-0.j,  1.+0.j, -0.+0.j],
       [-0.+0.j, -0.+0.j,  1.+0.j]])

In [9]:
b = Zabc
b

array([[ 0.8667+2.0417j,  0.2953+0.9502j,  0.2907+0.729j ],
       [ 0.2953+0.9502j,  0.8837+1.9852j,  0.2992+0.8023j],
       [ 0.2907+0.729j ,  0.2992+0.8023j,  0.8741+2.0172j]])

In [10]:
c = 0.5 * np.dot(Yabc*1e-6, U+a)
c.round(2)

array([[-0.+0.j,  0.-0.j,  0.-0.j],
       [ 0.-0.j, -0.+0.j,  0.-0.j],
       [ 0.-0.j,  0.-0.j, -0.+0.j]])

In [11]:
d = U + 0.5 * np.dot(Yabc*1e-6, b)
d.round(2)

array([[ 1.+0.j, -0.-0.j, -0.+0.j],
       [-0.-0.j,  1.+0.j, -0.+0.j],
       [-0.+0.j, -0.+0.j,  1.+0.j]])

## Observações a respeito da matriz de admitâncias em derivação

Como é possível observar, como a matriz de admitância tem valores muito baixos, a matriz **a** pode ser representada como a matriz identidade e a matriz **c** como uma matriz de zeros. Veja:

In [12]:
a[0,0]

(0.99999117323072706+3.9472767985250234e-06j)

In [13]:
a[0,1]

(-1.0213609819788794e-06-1.574878759182738e-07j)

In [14]:
c[0,0]

(-2.1153960902270234e-11+1.0740675019044989e-05j)

O fato aqui é que nesses casos a matriz de admitâncias pode ser desprezada, pois seu efeito é muito pequeno no sistema.

## Cálculo das tensões no nó m

Calculando a tensão fase-neutro do sistema:

In [15]:
van = 12470.0 / np.sqrt(3)
van

7199.5579

Como a carga foi dita equilibrada podemos montar a matriz de tensões em **m**:

In [16]:
Vabcm = np.array([p(van, 0.0), p(van, -120.0), p(van, 120.0)])
Vabcm = np.reshape(Vabcm, (3, 1))
Vabcm

array([[7199.56 ∠ 0.0º],
       [7199.56 ∠ -120.0º],
       [7199.56 ∠ 120.0º]], dtype=object)

## Cálculo das correntes no nó m

Agora a corrente será de:

In [17]:
ia = 6e6 / (np.sqrt(3) * 12.47e3)
ia

277.7948

Como o fator de potência é de 0,9 e indutivo:

In [18]:
angle = - np.arccos(0.9) * 180.0 / np.pi 
angle

-25.8419

In [19]:
Iabcm = np.array([p(ia, angle), p(ia, -120.0+angle), p(ia, 120.0+angle)])
Iabcm = np.reshape(Iabcm, (3, 1))
Iabcm

array([[277.79 ∠ -25.84º],
       [277.79 ∠ -145.84º],
       [277.79 ∠ 94.16º]], dtype=object)

## Cálculo das tensões no nó n

Aplicando a equação da tensão no nó **n**:

\begin{equation}
        \mathbf{V_{abc}}_n = \mathbf{a} \cdot \mathbf{V_{abc}}_m + \mathbf{b} \cdot  \mathbf{I_{abc}}_m 
\end{equation}

In [20]:
Vabcn = np.dot(a, Vabcm)  + np.dot(b, Iabcm)
Vabcn[0,0].polar = True
Vabcn[1,0].polar = True
Vabcn[2,0].polar = True

Vabcn

array([[7538.74 ∠ 1.57º],
       [7451.25 ∠ -118.3º],
       [7485.11 ∠ 121.93º]], dtype=object)

## Cálculo das tensões em pu

In [21]:
Vabcn_pu = 1.0 / van * Vabcn
Vabcn_pu[0,0].polar = True
Vabcn_pu[1,0].polar = True
Vabcn_pu[2,0].polar = True
Vabcn_pu

array([[1.05 ∠ 1.57º],
       [1.03 ∠ -118.3º],
       [1.04 ∠ 121.93º]], dtype=object)

## Cálculo das correntes no nó n

A corrente no nó *m* é facilmente encontrada, aplicando:

\begin{equation}
        \mathbf{I_{abc}}_n = \mathbf{c} \cdot \mathbf{V_{abc}}_m +  \mathbf{d} \cdot \mathbf{I_{abc}}_m
\end{equation}

In [22]:
Iabcn = np.dot(c, Vabcm)  + np.dot(d, Iabcm)
Iabcn[0,0].polar = True
Iabcn[1,0].polar = True
Iabcn[2,0].polar = True

Iabcn

array([[277.74 ∠ -25.83º],
       [277.76 ∠ -145.82º],
       [277.76 ∠ 94.17º]], dtype=object)